<a href="https://colab.research.google.com/github/flying-bear/kompluxternaya/blob/master/assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [0]:
!п

In [0]:
import nltk
import numpy as np
import re

from collections import Counter
from google.colab import drive
from sklearn.base import TransformerMixin


In [9]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
root_path = 'gdrive/My Drive/studies/HSE/prog/kompluxternaya'
with open(root_path+'/'+'peace.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [11]:
def preprocess_text(text):
    # TODO
    # make lowercase
    text = text.lower()
    # replace all punctuation except dots with spaces
    pattern = '!|"|#|\$|%|&|\'|\(|\)|\*|\+|,|-|/|:|;|<|=|>|\?|@|\[|\\|\]|^|_|`|{|\||}|~|”|“|—|‘|’'
    text = re.sub(pattern, ' ', text)
    # collapse multiple spaces into one '   ' -> ' '
    text = re.sub('\s+', ' ', text)
    return text

text = preprocess_text(text)
#assert len(text) == 3141169
len(text)

3141171

In [0]:
text = text.split('.')
text = [x.strip() for x in text]

In [0]:
def list_to_bigrams(l):
  """
  takes a list of lists of integers or strings
  returns a counter of their pairs (tuples) across the lists
  """
  bigrams = Counter()
  for i in range(len(l)):
    bigrams.update((x, y) for x, y in zip(*[l[i][j:] for j in range(2)]))
  return bigrams

In [47]:
t = [[1, 2, 3, 4, 1, 2], [1, 2], [3,4], [2]]
new_token = (1,2)
new_id = 5
t_new = t.copy()
for i, sent in enumerate(t):
  for j, (v, w) in enumerate(zip(sent[:-1], sent[1:])):
    if new_token == (v, w):
      t_new[i][j] = new_id
      del t_new[i][j-1]
print(t_new)

[[5, 2, 3, 5], [5], [3, 4], [2]]


In [0]:
class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}

    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        # tokenize text by symbols and fill in self.itos and self.stoi
        self.itos = list(set(text)) # list letters
        self.stoi = dict(zip(list(set(text)), range(len(text)))) # dict of letters and their ids
        text = [self.itos.index(x) for x in text[i]] for i in range(len(text))] # replace letters with thir ids (str to list)
        
        while len(self.itos) < self.vocab_size:
            bigrams = list_to_bigrams(text) # get text bigrams
            new_token = bigrams.most_common(1)[0][0] # find most common bigram
            new_id = len(self.itos)
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            # find occurences of the new_token in the text and replace them with new_id
            text_new = text.copy()
            for i, sent in enumerate(text):
              for j, (v, w) in enumerate(zip(sent[:-1], sent[1:])):
                if new_token == (v, w):
                  text_new[i][j] = new_id
                  del text_new[i][j-1]
            text = text_new
            
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        text = # TODO tokenize text by symbols using self.stoi
        for token_id, token in enumerate(self.itos):
            # find occurences of the token in the text and replace them with token_id
            text = # TODO       
        return text
    
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        result = # TODO
        return result
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
        
        
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

SyntaxError: invalid syntax (<ipython-input-12-0c09a6b70534>, line 25)

In [0]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [0]:
start_token = vocab_size
end_token = vocab_size + 1
        
    
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = # TODO create array for storing 3-gram counters
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = # TODO
        return result
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        result = # TODO approximate probability by counters
        return result
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        self.proba = # TODO count 3-grams in the text
        
        return self
    
lm = LM(vocab_size, 1).fit(tokenized_text)

In [0]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    
    beam = # TODO store in beam tuples of current sequences and their log probabilities
    
    for i in range(max_len):
        candidates = []
        candidates_proba = []
        for snt, snt_proba in beam:
            if # TODO process terminal token
            else:    
                proba = # probability vector of the next token
                best_k = # top-k most probable tokens
                # TODO update candidates' sequences and corresponding probabilities
                
        beam = # select top-k most probable sequences from candidates
    return beam
    

In [0]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
    

In [0]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

In [0]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
# TODO print decoded generated strings and their probabilities

In [0]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

In [0]:
def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    result = #TODO perplexity for the sentence
    return result

perplexity(tokenized_text[0], lm)